In [ ]:
#Installing libraries

#main.py
!pip install -q streamlit openai-whisper
!pip install -q streamlit-audio-recorder
!pip install -q pyttsx3
!pip install -q python-dotenv
!pip install audio_recorder_streamlit
!pip install gTTS
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/openai/whisper.git

#emotion analyzer
!pip install SpeechRecognition transformers torch
!pip install tf-keras

#llm agent
!pip install langchain openai langgraph langsmith
!pip install python-dotenv
!pip install langchain.llms
!pip install langchain.prompts
!pip install langchain.chains
!pip install langchain.memory
!pip install langgraph
!pip install langchain_community
!pip install langchain_experimental
!pip install --upgrade langchain
!pip install --upgrade langsmith
!pip install ollama_client
!pip install langgraph
!pip install ollama
!pip install -q openai


ERROR: Could not find a version that satisfies the requirement streamlit-audio-recorder (from versions: none)
ERROR: No matching distribution found for streamlit-audio-recorder
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-b9j5dueq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-b9j5dueq
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802825 sha256=1ced73a218bdbe6222f2d6955f57ac1e8eb86066cf3471984daf0e0fef2cd931
  Stored in directory: /tmp/pip-ephem-wheel-cache-q19661kn/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
  Attempting uninstall: openai-whisper
    Found existing 

In [ ]:
%%writefile app.py

#Importing libraries

#main.py
import os
import sys
import datetime
import openai
import dotenv
import streamlit as st
#from openai import OpenAI
from audio_recorder_streamlit import audio_recorder
import pyttsx3
from io import BytesIO
from gtts import gTTS
import whisper
import logging

#emotion analyzer
import speech_recognition as sr
from transformers import pipeline

#llm agent
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langgraph.graph import Graph
from langchain.smith import RunEvalConfig, run_on_dataset
from langsmith import Client
from langchain.callbacks.tracers import LangChainTracer
from langchain.llms import Ollama
from langgraph.graph import StateGraph
from typing_extensions import TypedDict
import os



#llm agent
#set Enviroment

os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_ENDPOINT"] = "<lamg_chain_url>"
os.environ["LANGCHAIN_API_KEY"] = "<langsmith_api_key>"
os.environ["OPENAI_API_KEY"] = "<open_ai_key>"


#Load environment variables
dotenv.load_dotenv()

#client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
#client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

model = whisper.load_model("base")
  # It's better to use environment variables for API keys

# Initialize the OpenAI client
#client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

#def text_to_speech(text):
#    engine = pyttsx3.init()
#    engine.save_to_file(text, 'response.mp3')
#    engine.runAndWait()
#


#llm agent


#Bot Config

summary_config = [{"Current Summary": "", "Prompt": "Hello, I'm PsyOzen AI. How can I help you today?", "Text": "", "New Summary": "There doesnt seem to be enough imformation for new summary. Need to ask user to say something in a polite way."}, { "Current Summary": "First Message", "Prompt": "Hello, I'm PsyOzen AI. How can I help you today?", "Text": "Hi, I'm Tom. My wife Emma and I are seriously considering divorce after 15 years of marriage. We're both at our wit's end.", "New Summary": "Tom and his wife Emma are contemplating divorce after 15 years of marriage. The situation appears dire." }, { "Current Summary": "Tom and his wife Emma are contemplating divorce after 15 years of marriage. The situation appears dire.", "Prompt": "I understand you're going through a difficult time, Tom. We will focus on how our thoughts affect our feelings and behaviors. Can you identify a specific situation recently that led to conflict between you and Emma?", "Text": "Last night, Emma asked me about my day at work. I gave a short answer because I was tired. She then said, 'You never want to talk to me anymore.' I felt attacked and just walked away. We ended up not speaking for the rest of the evening.", "New Summary": "Tom describes a recent conflict where his short response to Emma's question about his day led to her expressing frustration about his communication. Tom felt attacked and withdrew, resulting in an evening of silence. This situation demonstrates negative thought patterns and behaviors that CBT can address." }, { "Current Summary": "Tom describes a recent conflict where his short response to Emma's question about his day led to her expressing frustration about his communication. Tom felt attacked and withdrew, resulting in an evening of silence. This situation demonstrates negative thought patterns and behaviors that CBT can address.", "Prompt": "Thank you for sharing that, Tom. Let's break this down. What thoughts went through your mind when Emma said, 'You never want to talk to me anymore'?", "Text": "I thought, 'She's always criticizing me. Nothing I do is ever good enough for her. She doesn't understand how hard I work.' It made me feel angry and unappreciated.", "New Summary": "Tom's automatic thoughts in response to Emma's comment include beliefs about constant criticism, inadequacy, and lack of understanding. These thoughts led to feelings of anger and being unappreciated, resulting in withdrawal behavior. This cognitive pattern is a key focus for CBT intervention." }, { "Current Summary": "Tom's automatic thoughts in response to Emma's comment include beliefs about constant criticism, inadequacy, and lack of understanding. These thoughts led to feelings of anger and being unappreciated, resulting in withdrawal behavior. This cognitive pattern is a key focus for CBT intervention.", "Prompt": "I see. Now, let's examine these thoughts. Are they factually true? Are there any exceptions to 'She's always criticizing me' or 'Nothing I do is ever good enough'?", "Text": "Well, when you put it that way... I guess she does compliment me sometimes, especially about how I am with the kids. And she was supportive when I got that promotion last year. Maybe 'always' and 'nothing' are too strong.", "New Summary": "Through CBT questioning, Tom recognizes that his automatic thoughts contain cognitive distortions, specifically overgeneralization. He acknowledges exceptions to his beliefs about constant criticism and inadequacy. This realization opens the door for cognitive restructuring and behavior modification." } ]


response_config = [{"Current Summary": "There doesnt seem to be enough imformation for new summary. Need to ask user for input in a polite way.", "Text": "", 'Response': "It seems you haven't shared the purpose of your presence here today."}, {'Current Summary': "Tom and his wife Emma are contemplating divorce after 15 years of marriage. The situation appears critical and requires CBT intervention.", 'Text': "Hi, I'm Tom. My wife Emma and I are seriously considering divorce after 15 years of marriage. We're both at our wit's end.", 'Response': "I understand you're going through a difficult time, Tom. We will focus on how our thoughts affect our feelings and behaviors. This approach can be very helpful in addressing relationship issues. To start, it would be helpful to examine a specific situation that led to conflict between you and Emma." }, { 'Current Summary': "Tom describes a recent conflict where his short response to Emma's question about his day led to her expressing frustration about his communication. Tom felt attacked and withdrew, resulting in an evening of silence. This situation demonstrates negative thought patterns and behaviors that CBT can address.", 'Text': "Last night, Emma asked me about my day at work. I gave a short answer because I was tired. She then said, 'You never want to talk to me anymore.' I felt attacked and just walked away. We ended up not speaking for the rest of the evening.", 'Response': "Thank you for sharing that specific example. Let's break this down into thoughts, feelings, and behaviors. You've described your behavior (giving a short answer and walking away) and Emma's response. Now, let's focus on your thoughts. What went through your mind when Emma said, 'You never want to talk to me anymore'?" }, { 'Current Summary': "Tom's automatic thoughts in response to Emma's comment include beliefs about constant criticism, inadequacy, and lack of understanding. These thoughts led to feelings of anger and being unappreciated, resulting in withdrawal behavior. This cognitive pattern is a key focus for CBT intervention.", 'Text': "I thought, 'She's always criticizing me. Nothing I do is ever good enough for her. She doesn't understand how hard I work.' It made me feel angry and unappreciated.", 'Response': "Thank you for identifying those thoughts, Tom. We call these 'automatic thoughts' - they pop into our minds quickly and can strongly influence our emotions and actions. Now, let's examine these thoughts more closely. One technique we usually use to question the validity of our automatic thoughts. Are these thoughts factually true? Are there any exceptions to 'She's always criticizing me' or 'Nothing I do is ever good enough'?" }, { 'Current Summary': "Through CBT questioning, Tom recognizes that his automatic thoughts contain cognitive distortions, specifically overgeneralization. He acknowledges exceptions to his beliefs about constant criticism and inadequacy. This realization opens the door for cognitive restructuring and behavior modification.", 'Text': "Well, when you put it that way... I guess she does compliment me sometimes, especially about how I am with the kids. And she was supportive when I got that promotion last year. Maybe 'always' and 'nothing' are too strong.", 'Response': "Excellent observation, Tom. You've just engaged in what we call 'cognitive restructuring' - challenging and modifying unhelpful thought patterns. You've identified that words like 'always' and 'nothing' are overgeneralizations. This is a common cognitive distortion. By recognizing these exceptions, you're starting to see a more balanced view of your relationship with Emma. Now, let's think about how this realization might change your feelings and behaviors in similar situations in the future." } ]


question_config = [{'Text': "", 'Response': "It seems you haven't you havent shared an input.", 'Question': "How can I help you today?" },{'Text': "Hi PysOzen, I'm Tom. My wife Emma and I are seriously considering divorce after 15 years of marriage. We're both at our wit's end.", 'Response': "I understand you're going through a difficult time, Tom. In situations like these, we focus on how our thoughts affect our feelings and behaviors. This approach can be very helpful in addressing relationship issues.", 'Question': "To start, can you identify a specific situation recently that led to conflict between you and Emma?" }, { 'Text': "Last night, Emma asked me about my day at work. I gave a short answer because I was tired. She then said, 'You never want to talk to me anymore.' I felt attacked and just walked away. We ended up not speaking for the rest of the evening.", 'Response': "Thank you for sharing that specific example, Tom. Let's break this down into thoughts, feelings, and behaviors. You've described your behavior (giving a short answer and walking away) and Emma's response.", 'Question': "Now, let's focus on your thoughts. What went through your mind when Emma said, 'You never want to talk to me anymore'?" }, { 'Text': "I thought, 'She's always criticizing me. Nothing I do is ever good enough for her. She doesn't understand how hard I work.' It made me feel angry and unappreciated.", 'Response': "Thank you for identifying those thoughts, Tom. We refer to these as 'automatic thoughts' - they pop into our minds quickly and can strongly influence our emotions and actions. Now, let's examine these thoughts more closely. One technique we use in situations like these is to question the validity of our automatic thoughts.", 'Question': "Are these thoughts factually true? Are there any exceptions to 'She's always criticizing me' or 'Nothing I do is ever good enough'?" }, { 'Text': "Well, when you put it that way... I guess she does compliment me sometimes, especially about how I am with the kids. And she was supportive when I got that promotion last year. Maybe 'always' and 'nothing' are too strong.", 'Response': "Excellent observation, Tom. You've just engaged in what we call 'cognitive restructuring' - challenging and modifying unhelpful thought patterns. You've identified that words like 'always' and 'nothing' are overgeneralizations. This is a common cognitive distortion. By recognizing these exceptions, you're starting to see a more balanced view of your relationship with Emma.", 'Question': "Now, let's think about how this realization might change your feelings and behaviors in similar situations in the future. How do you think you might respond differently next time Emma asks about your day?" } ]


recommendation_config = [{"User":["Hi PysOzen, I'm Tom. My wife Emma and I are seriously considering divorce after 15 years of marriage.", "Last night, Emma asked me about my day at work. I gave a short answer because I was tired. She then said, 'You never want to talk to me anymore.' I felt attacked and just walked away.", "I thought, 'She's always criticizing me. Nothing I do is ever good enough for her. She doesn't understand how hard I work.' It made me feel angry and unappreciated.", "Well, when you put it that way... I guess she does compliment me sometimes, especially about how I am with the kids. And she was supportive when I got that promotion last year. Maybe 'always' and 'nothing' are too strong.", "I could try to give a more detailed answer about my day, even if I'm tired. And maybe explain that I'm feeling drained instead of just walking away."], "Assistant": ["I understand you're going through a difficult time, Tom. We will focus on how our thoughts affect our feelings and behaviors. Can you identify a specific situation recently that led to conflict between you and Emma?", "Thank you for sharing that example. What thoughts went through your mind when Emma said, 'You never want to talk to me anymore'?", "Let's examine these thoughts. Are they factually true? Are there any exceptions to 'She's always criticizing me' or 'Nothing I do is ever good enough'?", "Excellent observation, Tom. You've just engaged in cognitive restructuring. Now, let's think about how this realization might change your feelings and behaviors in similar situations in the future. How do you think you might respond differently next time Emma asks about your day?"], "Recommendation": "Tom, based on our discussion, here are some recommendations to help you apply these insights to your relationship:1. Thought Record: Keep a daily thought record. When you experience strong negative emotions, write down the situation, your automatic thoughts, and your emotional response. Then, challenge these thoughts and write down more balanced alternatives.2. Cognitive Restructuring Practice: Continue to question your automatic thoughts, especially those involving absolutes like 'always' or 'never'. Look for evidence that contradicts these thoughts.3. Behavioral Experiment: Test your beliefs. For instance, for one week, give more detailed responses about your day and express when you're feeling tired. Observe Emma's reactions and compare them to your predictions.4. Communication Skills Training: Practice using 'I' statements to express your feelings without blaming. For example, 'I feel overwhelmed when I come home from work' instead of 'You don't understand how hard I work'.5. Mindfulness Exercises: Incorporate mindfulness techniques to help you stay present and respond thoughtfully rather than react automatically in stressful situations.6. Positive Activity Scheduling: Plan and engage in activities you both enjoy. This can help create positive experiences and challenge the belief that 'nothing is ever good enough'.7. Gratitude Journal: Each day, write down one thing you appreciate about Emma. This can help balance negative thoughts and improve your overall perception of the relationship.8. Regular Check-ins: Schedule short, daily check-ins with Emma where you both share your thoughts and feelings. This can improve communication and challenge the belief that you 'never want to talk'.Remember, changing thought patterns and behaviors takes time and practice. Be patient with yourself and celebrate small improvements. If you find these techniques helpful, consider seeking a couples therapist who can guide you both through this process more comprehensively." } ]


## Bot config prompt strings

summary_kb = "\n".join([f"Example {i+1}:\nCurrent Summary: {item['Current Summary']}\nPrompt: {item['Prompt']}\nText: {item['Text']}\nNew Summary: {item['New Summary']}" for i, item in enumerate(summary_config)])
response_kb = "\n".join([f"Example {i+1}:\nCurrent Summary: {item['Current Summary']}\nText: {item['Text']}\nResponse: {item['Response']}" for i, item in enumerate(response_config)])
question_kb = "\n".join([f"Example {i+1}:\nText: {item['Text']}\nResponse: {item['Response']}\nQuestion: {item['Question']}" for i, item in enumerate(question_config)])
recommendation_kb = "\n".join([f"Example {i+1}:\nUser: {item['User']}\nAssistant: {item['Assistant']}\nRecommendation: {item['Recommendation']}" for i, item in enumerate(recommendation_config)])

## Prompt templates

summary_prompt = PromptTemplate(
    input_variables=["current_summary", "prompt", "text", "emotion"],
    template=f"Use the following examples as a guide to generate a new summary and pass it to response:\n{summary_kb} based on identified emotions {{emotion}}.\nCurrent Summary: {{current_summary}}\nPrompt: {{prompt}}\nText: {{text}}\nNew Summary:"
)


response_prompt = PromptTemplate(
    input_variables=["current_summary", "text", "emotion"],
    template=f"""The following is a conversation with an AI therapist. The therapist is empathetic, helpful, creative, clever, and very friendly. Use these examples as a guide:

{response_kb}

Now, generate a response for:
Current Summary: {{current_summary}}
Text: {{text}}
Identofied emotions: {{emotion}} Please me mindful and empatheic while generating the response.
Response:"""
)

question_prompt = PromptTemplate(
    input_variables=["text", "response", "emotion"],
    template=f"""Use these examples as a guide to generate a follow-up question considering {{emotion}} too:

{question_kb}

Now, generate a question for:
Text: {{text}}
Response: {{response}}
Please me mindful and empatheic while genrating a question.
Question:"""
)

recommendation_prompt = PromptTemplate(
    input_variables=["user", "assistant"],
    template=f"""Following is a sample of conerastion with corresponding recommendation:

{recommendation_kb}

Now, generate a recommendation for the following conversation:
User: {{user}}
Assistant: {{assistant}}
Recommendation:"""
)


#llm agent
#Create LLMChain
#Open AI LLM
#Ollma LLM
llm = Ollama(model="stablelm2", base_url="<local_url>")


## With LangSMith Tracer

summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
response_chain = LLMChain(llm=llm, prompt=response_prompt)
question_chain = LLMChain(llm=llm, prompt=question_prompt)
recommendation_chain = LLMChain(llm=llm, prompt=recommendation_prompt)


# llm agent

# Process function

def process_input(state, input, emotions):
    # Generate summary
    summary = summary_chain.run(current_summary=state['summary'], prompt=state['last_response'], text=input, emotion = str(emotions))
    # Generate response
    response = response_chain.run(current_summary=summary, text=input, emotion = str(emotions))
    # Generate question if needed
    if not response.endswith('?'):
        question = question_chain.run(text=input, response=response, emotion = str(emotions))
        response += ' ' + question

    # Update state
    state['summary'] = summary
    state['last_response'] = response
    state['conversation'].append({"role": "user", "content": input})
    state['conversation'].append({"role": "assistant", "content": response})

    return state, response

#Llm agent
#Condition function

def should_continue(state):
    # Logic to determine if the conversation should continue
    return len(state['conversation']) < 20  # For example, continue for 10 turns

#llm agent
#Generate recommendations

def generate_recommendation(state):
    user_convo_list = []
    assistant_convo_list = []

    for message in state['conversation']:
        if message['role'] == 'user':
            user_convo_list.append(message['content'])
        elif message['role'] == 'assistant':
            assistant_convo_list.append(message['content'])




    recommendation = recommendation_chain.run(user=user_convo_list, assistant=assistant_convo_list)


    state['conversation'].append({"role": "user", "content": "Generate Recommendation"})
    state['conversation'].append({"role": "assistant", "content": recommendation})
    return state, recommendation


## llm agent
#Lang Graph config

# Define the graph

# Define the state schema
class ChatState(TypedDict):
    summary: str
    last_response: str
    conversation: list

workflow = StateGraph(state_schema=ChatState)

workflow.add_node("process_input", process_input)
workflow.add_node("should_continue", should_continue)
workflow.add_node("generate_recommendation", generate_recommendation)


# Modify the 'should_continue' function to set the 'next_step'
def should_continue(state):
    if len(state['conversation']) < 20:
        return "process_input"
    else:
        return "generate_recommendation"

#Use add_edge_with_condition for branching
workflow.set_entry_point("process_input")
workflow.add_edge("process_input", "should_continue")
workflow.add_conditional_edges(
    "should_continue",
    {
        "process_input": process_input,
        "generate_recommendation": generate_recommendation
    }
)

workflow.set_finish_point("generate_recommendation")
#workflow.add_edge("process_input", "should_continue")
#workflow.add_edge("should_continue", "process_input")
#workflow.add_edge("should_continue", "generate_recommendation")

#llm agent
#Compile langgraph config

# Compile the graph
app = workflow.compile()

#Emotion analyzer
# Function to analyze emotion from text
def analyze_emotion(text):
  emotion_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
  emotions = emotion_analyzer(text)
  return emotions

def transcribe(file_name):
  #audio_file= open(file_name, "rb")
  #transcription = client.audio.transcriptions.create(
  #model="whisper-1",
  #file=audio_file)
  result = model.transcribe(file_name)
  transcript = result["text"]
  return transcript

def save_audio_file(audio_bytes, file_extension):
  timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
  file_name = f"audio_{timestamp}.{file_extension}"
  with open(file_name, "wb") as f:
    f.write(audio_bytes)
  return file_name

def text_to_speech(text):
  tts = gTTS(text=text, lang='en')
  tts.save("response.mp3")
  # Play the audio as before


def main():

    # Configure logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    st.title("PsyOzen AI Voice Chat Interface")

    # Initialize session state
    if 'state' not in st.session_state:
      st.session_state.state = {
          'summary': "",
          'last_response': "Hello, I'm PsyOzen AI. How can I help you today?",
          'conversation': []
      }
      # Display chat messages from history
    for message in st.session_state.state['conversation']:
        if type(message) == dict:

          with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # Accept user input
    if prompt := st.chat_input("Hello, I'm PsyOzen AI. How can I help you today?"):
        # Add user message to chat history
        st.session_state['conversation'].append({"role": "user", "content": prompt})
        # Display user message in chat message container
        with st.chat_message("user"):
            st.markdown(prompt)




    # Record audio input
    audio_bytes = audio_recorder(pause_threshold=5.0)
    if audio_bytes:
        st.audio(audio_bytes, format="audio/wav")
        file_name = save_audio_file(audio_bytes, "wav")

        # Transcribe audio
        # Transcribe audio
        #with open(file_name, "rb") as audio_file:
        transcript = transcribe(file_name)
          #Emotion analyzer

        emotions = analyze_emotion(transcript)

        # Process user input
        st.session_state.state, response = process_input(st.session_state.state, transcript, emotions)

        # Add user message and AI response to conversation
        #st.session_state.state['conversation'].append({"role": "user", "content": transcript})
        #st.session_state.state['conversation'].append({"role": "assistant", "content": response})

        # Display transcribed message
        with st.chat_message("user"):
            st.markdown(transcript)

        # Display and speak AI response
        with st.chat_message("assistant"):
            st.markdown(response)
            text_to_speech(response)
            audio_file = open('response.mp3', 'rb')
            audio_bytes = audio_file.read()
            st.audio(audio_bytes, format='audio/mp3')

    # Check if we should continue or generate recommendation
        next_step = should_continue(st.session_state.state)

        if next_step == "generate_recommendation":

            st.session_state.state, recommendation = generate_recommendation(st.session_state.state)


            with st.chat_message("assistant"):
              st.markdown(recommendation)

              text_to_speech(recommendation)
              audio_file = open('response.mp3', 'rb')
              audio_bytes = audio_file.read()
              st.audio(audio_bytes, format='audio/mp3')



        # Clean up temporary files
        os.remove(file_name)
        os.remove('response.mp3')

        # Rerun to update the display


if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.193.32.67


In [ ]:
!npm install -g localtunnel


changed 22 packages, and audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!streamlit run app.py & pgrep streamlit & npx localtunnel --port 8501

47642



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.193.32.67:8501

your url is: https://many-laws-rule.loca.lt
2024-08-28 06:04:47.251781: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 06:04:47.278302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 06:04:47.286814: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 06:04:47.306782: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU in